# Numpy Memory Array for Fast Image Loading

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
from pathlib import Path

import numpy as np
from PIL import Image

from model.memory_map import MemMapReader, MemMapWriter

## Writer

In [61]:
image_paths = Path('outputs/speed_test').glob('*.png')
mm_writer = MemMapWriter(image_paths, 'outputs/mm/mem_map.dat')
mm_writer.write()
print(mm_writer)

Saving memmap: 100%|██████████| 476/476 [00:00<00:00, 2297.96it/s]


## Reader

In [34]:
mm_reader = MemMapReader('outputs/speed_test3/mm2/mem_map.dat', (256, 256))

In [30]:
len(mm_reader)

476

In [60]:
mm_reader[len(mm_reader) - 1].shape

(256, 256)

### Memory Array

In [53]:
%%timeit

for img in mm_reader:
    x = img.shape

355 μs ± 20.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [55]:
%%timeit

for img in mm_reader.iter_windows(4):
    x = (len(img), img[0].shape)

1.53 ms ± 64.8 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Pillow

In [54]:
%%timeit

for img_path in Path('outputs/speed_test3').glob('*.png'):
    x = np.array(Image.open(img_path)).shape

112 ms ± 5.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [59]:
%%timeit

image_paths = list(Path('outputs/speed_test3').glob('*.png'))
for i in range(0, len(image_paths), 4):
    xs = [np.array(Image.open(img_path)) for img_path in image_paths[i : i + 4]]
    x = (len(xs), xs[0].shape)

115 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
